In [1]:
%matplotlib inline
import stac_ipyleaflet
import importlib
importlib.reload(stac_ipyleaflet)

<module 'stac_ipyleaflet' from '/Users/aimeebarciauskas/github/maap-project/stac_ipyleaflet/stac_ipyleaflet/__init__.py'>

# Use ipyleaflet to visually explore MAAP data

Inspired by leafmap.

In [2]:
import ipywidgets
m = stac_ipyleaflet.StacIpyleaflet(zoom=4, layout=ipywidgets.Layout(height="800px"))
m.draw_biomass_map()
m

StacIpyleaflet(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [29]:
import ipyleaflet
visible_layers = [layer for layer in m.layers if type(layer) == ipyleaflet.TileLayer and layer.visible and not layer.base]

In [30]:
layer_url = visible_layers[0].url

In [31]:
layer_url

'https://titiler.maap-project.org/mosaicjson/mosaics/f0e6f941-4e50-46b3-9463-11b3c2181662/tiles/{z}/{x}/{y}?bidx=1&max_size=1024&resampling_method=nearest&rescale=0,400&return_mask=true&colormap_name=gist_earth_r'

In [32]:
import re
match = re.search('(https://.+)/tiles', layer_url)

In [33]:
mosaic_url = match.groups()[0]
mosaic_url

'https://titiler.maap-project.org/mosaicjson/mosaics/f0e6f941-4e50-46b3-9463-11b3c2181662'

In [34]:
from shapely.geometry import Polygon
geometries = [m.draw_control.last_draw['geometry']]
if geometries[0]:
    box = Polygon(geometries[0]['coordinates'][0])
    bounds = box.bounds
str_bounds = f"{bounds[0]},{bounds[1]},{bounds[2]},{bounds[3]}"

In [35]:
assets_endpoint = f"{m.titiler_url}/mosaicjson/{str_bounds}/assets?url={mosaic_url}/mosaicjson"

In [36]:
import requests
assets_response = requests.get(assets_endpoint)
assets = assets_response.json()

In [37]:
assets, str_bounds

(['s3://maap-landing-zone-gccops/user-added/uploaded_objects/e1a75192-4c17-4176-ab63-2010c368df13/global_008_06dc_agb_mean_prediction_2020_mosaic_veg_gfccorr_scale1_Europe_cog.tif',
  's3://maap-landing-zone-gccops/user-added/uploaded_objects/e1a75192-4c17-4176-ab63-2010c368df13/global_008_06dc_agb_mean_prediction_2020_mosaic_veg_gfccorr_scale1_Asia_cog.tif',
  's3://maap-landing-zone-gccops/user-added/uploaded_objects/e1a75192-4c17-4176-ab63-2010c368df13/global_008_06dc_agb_mean_prediction_2020_mosaic_veg_gfccorr_scale1_Oceania_cog.tif',
  's3://maap-landing-zone-gccops/user-added/uploaded_objects/e1a75192-4c17-4176-ab63-2010c368df13/global_008_06dc_agb_mean_prediction_2020_mosaic_veg_gfccorr_scale1_Africa_cog.tif'],
 '11.918673,3.187137,14.554946,5.728044')

In [45]:
datasets = []
import numpy
from io import BytesIO
import xarray as xr
for asset in assets:
    crop_endpoint = f"{m.titiler_url}/cog/crop/{str_bounds}.npy?url={asset}&max_size=512"  # Same here you can either use max_size or width&height
    res = requests.get(crop_endpoint)
    arr = numpy.load(BytesIO(res.content))
    tile, mask = arr[0:-1], arr[-1]

    # TODO:
    # convert tile/mask to xarray dataset
    ds = xr.DataArray(tile)
    datasets.append(ds)

In [48]:
datasets

[<xarray.DataArray (dim_0: 1, dim_1: 494, dim_2: 512)>
 array([[[-32768, -32768, -32768, ..., -32768, -32768, -32768],
         [-32768, -32768, -32768, ..., -32768, -32768, -32768],
         [-32768, -32768, -32768, ..., -32768, -32768, -32768],
         ...,
         [-32768, -32768, -32768, ..., -32768, -32768, -32768],
         [-32768, -32768, -32768, ..., -32768, -32768, -32768],
         [-32768, -32768, -32768, ..., -32768, -32768, -32768]]],
       dtype=int16)
 Dimensions without coordinates: dim_0, dim_1, dim_2,
 <xarray.DataArray (dim_0: 1, dim_1: 494, dim_2: 512)>
 array([[[-32768, -32768, -32768, ..., -32768, -32768, -32768],
         [-32768, -32768, -32768, ..., -32768, -32768, -32768],
         [-32768, -32768, -32768, ..., -32768, -32768, -32768],
         ...,
         [-32768, -32768, -32768, ..., -32768, -32768, -32768],
         [-32768, -32768, -32768, ..., -32768, -32768, -32768],
         [-32768, -32768, -32768, ..., -32768, -32768, -32768]]],
       dtype=int

In [38]:
%%time
ds1 = m.gen_mosaic_dataset_crop(assets, str_bounds)

ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation

In [40]:
%%time
ds2 = m.gen_mosaic_dataset_reader(assets, bounds)

CPU times: user 280 ms, sys: 32.7 ms, total: 313 ms
Wall time: 1.01 s
